In [1]:
import numpy as np
import scipy.optimize as opt

In [2]:
class f(object):
    def __init__(self, solver, goal):
        self.__solver = solver
        if goal == "max" or goal == "min":
            self.__goal = goal
        else:
            raise Exception("goal deve ser 'max' ou 'min'.")
        self.__f_max = None
        self.__x_max = None
        self.__f_min = None
        self.__x_min = None

    @property
    def goal(self):
        return self.__goal

    @property
    def f_max(self):
        return self.__f_max

    @f_max.setter
    def f_max(self, f_max):
        self.__f_max = f_max

    @property
    def x_max(self):
        return self.__x_max

    @x_max.setter
    def x_max(self, x_max):
        self.__x_max = x_max

    @property
    def f_min(self):
        return self.__f_min

    @f_max.setter
    def f_min(self, f_min):
        self.__f_min = f_min

    @property
    def x_min(self):
        return self.__x_min

    @x_min.setter
    def x_min(self, x_min):
        self.__x_min = x_min

    def solve(self, x, inv=False):
        x = np.array(x)
        return self.__solver(x) * -1 if inv else self.__solver(x)

    def mu(self, x):
        x = np.array(x)
        if self.__goal == "max":
            return (self.solve(x) - self.__f_min) / (self.__f_max - self.__f_min)
        elif self.__goal == "min":
            return (self.__f_max - self.solve(x)) / (self.__f_max - self.__f_min)

In [3]:
class f_linear(f):
    def __init__(self, coefs, goal):
        self.__coefs = np.array(coefs)
        f.__init__(self, lambda x: np.dot(self.__coefs, x), goal)

    @property
    def coefs(self):
        return self.__coefs

In [4]:
f1 = f_linear([0.06, 0.53, 0.18, 0.18, 0.06], "max")
f2 = f_linear([25, 70, 60, 95, 45], "max")
f3 = f_linear([0, 32.5, 300, 120, 0], "min")
f4 = f_linear([0.1, 0.1, 0.11, 0.35, 0.33], "min")

In [5]:
A_eq = np.array([[1, 1, 1, 1, 1]])
b_eq = np.array([3000])
x0_bounds = (0, 850)
x1_bounds = (0, 220)
x2_bounds = (0, 1300)
x3_bounds = (0, 1615)
x4_bounds = (0, 700)

In [6]:
F = [f1, f2, f3, f4]
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds]

In [7]:
from scipy.optimize import linprog
import warnings

for f_ in F:
    res = linprog(f_.coefs * -1, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    if res.success:
        f_.f_max = -1 * res.fun
        f_.x_max = res.x
    else:
        warn("Otimização de {} mal sucedida.").format(f_)

In [35]:
for f_ in F:
    res = linprog(f_.coefs, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    if res.success:
        f_.f_min = res.fun
        f_.x_min = res.x
    else:
        warn("Otimização de {} mal sucedida.").format(f_)

In [11]:
def mu_D(x, F):
    x = np.array(x)
    return min([f_.mu(x) for f_ in F])

In [12]:
fora_do_limite = max([f_.f_max for f_ in F]) + 10000

In [30]:
def fun_para_diff_evol(x, F, A_eq, b_eq, fora_do_limite):
    A_eq = np.array(A_eq)
    x = np.array(x)*-1
    return mu_D(x, F) if np.dot(x, A_eq) == b_eq else fora_do_limite

In [31]:
args = (F, A_eq[0], b_eq, fora_do_limite)
opt.differential_evolution(fun_para_diff_evol, bounds, args=args)

     fun: 596562.5
 message: 'Optimization terminated successfully.'
    nfev: 156
     nit: 1
 success: True
       x: array([ 32.48393975,  47.14971031, 138.74519826, 294.72577509,
       372.0619696 ])

In [37]:
[print (f_.f_max) for f_ in F]

617.0
238725.0
586562.5
871.5999999999999


[None, None, None, None]

In [36]:
[print (f_.f_min) for f_ in F]

617.0
238725.0
586562.5
871.5999999999999


[None, None, None, None]